In [156]:
import pandas as pd
import numpy as np

In [157]:
# import the excel file into a dataframe
df_whole = pd.read_excel('Cruise Configuration - Nautilus - test 2020.xlsx', sheet_name = None)

# count how many sheets in the whole excel sheet
sheetCnt = len(df_whole.keys())

# create a list and initialize the list item in the list
# for storing the subsheets
df_sheet = []
for i in range(0,sheetCnt):
    df_sheet.append("df_{}".format(i))

In [158]:
# create a list and append in the node name for filling use
node_fill = []
for sheet in df_whole.keys():
    node_fill.append(sheet) 

In [159]:
# open the sheet seperately and store them seperately in the created list
for i in range(0,sheetCnt):
    df_sheet[i] = pd.read_excel('Cruise Configuration - Nautilus - test 2020.xlsx', sheet_name = i)

In [160]:
# set a variable for storing the current position
pos = 0

# create a dataframe for storing the deploy row
df_out = pd.DataFrame()

# loop through all sheets in the excel sheets
for sheet in df_sheet:
    # Drop unused columns
    # only keep column from column 1 to column 12
    # which is NODE to OPERATION
    sheet.drop(sheet.iloc[:, 13::], inplace = True, axis = 1)
        
    # Assign the column names
    # since the first row initially is empty
    sheet.columns = ['Node', 'Junction Box', 'Port','Communications','Cable to Connector Panel', 'Cable', 'IP Address',
                 'Instrument Category', 'Instrument', 'Serial Number', 'Device ID', 'Work Ticket', 'Operation']
    
    # Drop the first row
    sheet = sheet.drop(index=0)
    
    # ~~~~~~~~~~~~~~~~~~~deal with the sub ticket
    # Set up the intial fill-in value
    currPort = ''
    currCom = ''
    currCtCP = ''
    currCEI = ''
    currIP = ''
    currIC = ''
    currIns = ''
    currSN = ''
    currDI = ''
    currWT = ''
    currOp = ''
    
    # check in the port column, whether already see the port in strin already
    # initially set to false
    meetString = False
    
    # iterate the rows in the subsheet
    # and record the values when encouter to the first valid record
    for index, row in sheet.iterrows():
        if(isinstance(row['Port'], str)):
            currPort = row['Port']
            currCom = row['Communications']
            currCtCP = row['Cable to Connector Panel']
            currCEI = row['Cable']
            currIP = row['IP Address']
            currIC = row['Instrument Category']
            currIns = row['Instrument']
            currSN = row['Serial Number']
            currDI = row['Device ID']
            currWT = row['Work Ticket']
            currOp = row['Operation']
            break
            
    # iterate rhw rows in the subsheet and do the subtickts fill        
    for index, row in sheet.iterrows():
        # if meet a empty port cell and already seen a string port before
        # check whether this row is empty or not
        # if is not, fill in with current fill-in values
        if(meetString == True and (not isinstance(row['Port'], str))):
            
            isCom = isinstance(row['Communications'], str)
            isCtCP = isinstance(row['Cable to Connector Panel'], str)
            isCEI = isinstance(row['Cable'], str)
            isIP = isinstance(row['IP Address'], str)
            isIC = isinstance(row['Instrument Category'], str)
            isIns = isinstance(row['Instrument'], str)
            isSN = isinstance(row['Serial Number'], str)
            isDI = isinstance(row['Device ID'], str)
            isWT = isinstance(row['Work Ticket'], str)
            isOp = isinstance(row['Operation'], str)
            
            # check whether the current is empty or not
            # in other words, the possibility of being a subtickets
            isNotEmpty = isCom or isCtCP or isCEI or isIP or isIC or isIns or isSN or isDI or isWT or isOp
               
#        isNotEmpty = row['Communications'] != np.nan or row['Cable to Connector Panel'] != np.nan 
#        or row['Cable EXT ID'] != np.nan or row['IP Address'] != np.nan or row['IP Address'] != np.nan 
#        or row['Instrument Category'] != np.nan or row['Instrument'] != np.nan or row['Serial Number'] != np.nan 
#        or row['Device ID'] != np.nan or row['Work Ticket'] != np.nan or row['Operation'] != np.nan

            # if is not empty -> is a subtickets -> fill the empty cells using current fill-in values
            # note: the fill-in values is inherited from the last port-non-empty row
            if(isNotEmpty):
            
                # fill out the empty cells of children tickets
                row['Port'] = currPort
                        
                if(not isCom):
                    row['Communications'] = currCom
                
                #if row['Communications'] == np.nan:
                 #   row['Communications'] = currCom
                
                if (not isCtCP):
                    row['Cable to Connector Panel'] = currCtCP
                
                if (not isCEI):
                    row['Cable'] = currCEI
                
                if (not isIP):
                    row['IP Address'] = currIP
                
                if (not isIC):
                    row['Instrument Category'] = currIC
                
                if (not isIns):
                    row['Instrument'] = currIns
                
                if (not isSN):
                    row['Serial Number'] = currSN
                
                if (not isDI):
                    row['Device ID'] = currDI
                    
                # do not inherit the work ticket value
                #if (not isWT):
                    #row['Work Ticket'] = currWT
                
                if (not isOp):
                    row['Operation'] = currOp
                    
        # if the current row is a new ticket, assign the current row values to the fill-in values
        if(isinstance(row['Port'], str) and row['Port'] != currPort):
            meetString = True
            currPort = row['Port']
            currCom = row['Communications']
            currCtCP = row['Cable to Connector Panel']
            currCEI = row['Cable']
            currIP = row['IP Address']
            currIC = row['Instrument Category']
            currIns = row['Instrument']
            currSN = row['Serial Number']
            currDI = row['Device ID']
            currWT = row['Work Ticket']
            currOp = row['Operation']
                   
    junction_box_fill = ""
    # find the juction box fill
    for item in sheet['Junction Box']:
        thisType = isinstance(item, str) 
        if thisType:
            junction_box_fill = item
        
    # fill the Node column and Junction Box column            
    sheet['Node'] = sheet['Node'].fillna(node_fill[pos])
    sheet['Junction Box'] = sheet['Junction Box'].fillna(junction_box_fill)
    
    # Extract the deploy row from the current sheet and append the row into the output dataframe
    for index, row in sheet.iterrows():
        if(row["Operation"] == "Deploy"):
            df_out = df_out.append(row, ignore_index = True)[list(sheet)]
    
    #jump to the next sheet
    pos += 1        
    

In [161]:
df_out

,Node,Junction Box,Port,Communications,Cable to Connector Panel,Cable,IP Address,Instrument Category,Instrument,Serial Number,Device ID,Work Ticket,Operation
0,Cascadia,"JB-13, 2700 m","J3 (15V, 75W)","Serial, RS485",1048181-XX,Ext 429,10.136.68.130:4002,CTD,SBE 19 Plus V2,7033.0,23580.0,https://jira.oceannetworks.ca/browse/EN-3609,Deploy
1,Cascadia,"JB-13, 2700 m","J3 (15V, 75W)","Serial, RS485",1048181-XX,Ext 429,10.136.68.130:4002,DO,SBE 63 DO,7033.0,23580.0,EN-3610,Deploy
2,Cascadia,"JB-13, 2700 m","J5 (48V, 75W)",Serial,1048181-XX,"1093189-37 (EXT-xx), ODI-12-Receptacle to XSL-...",10.136.68.130:4003,ADCP 75 kHz,RDI Workhorse Long Ranger,16511.0,23006.0,https://jira.oceannetworks.ca/browse/EN-3580,Deploy
3,Cascadia,"JB-13, 2700 m","J10 (400V, 15A)",Ethernet,1048181-XX,TBD,NaN,BPR,"NRCan Bottom Pressure Recorder, SE",87.0,21502.0,https://jira.oceannetworks.ca/browse/EN-3826,Deploy
4,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone Array,icLink,NaN,50320.0,https://jira.oceannetworks.ca/browse/EN-3926,Deploy
5,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320.0,NaN,Deploy
6,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320.0,NaN,Deploy
7,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320.0,NaN,Deploy
8,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320.0,NaN,Deploy
9,Clayoquot,JB-08,J10 (400V 15A),Ethernet,1048181-xx,"1093189-xx, EXT-xx ,70m (already on seafloor) ...",NaN,Hydrophone Array,icLink,4042.0,48940.0,https://jira.oceannetworks.ca/browse/EN-3924,Deploy


In [162]:
# insert the column needed in the jira import tool
df_out.insert(11, "Component", "Test and Development")
df_out.insert(13, "Linked To", np.nan)
df_out

,Node,Junction Box,Port,Communications,Cable to Connector Panel,Cable,IP Address,Instrument Category,Instrument,Serial Number,Device ID,Component,Work Ticket,Linked To,Operation
0,Cascadia,"JB-13, 2700 m","J3 (15V, 75W)","Serial, RS485",1048181-XX,Ext 429,10.136.68.130:4002,CTD,SBE 19 Plus V2,7033.0,23580.0,Test and Development,https://jira.oceannetworks.ca/browse/EN-3609,NaN,Deploy
1,Cascadia,"JB-13, 2700 m","J3 (15V, 75W)","Serial, RS485",1048181-XX,Ext 429,10.136.68.130:4002,DO,SBE 63 DO,7033.0,23580.0,Test and Development,EN-3610,NaN,Deploy
2,Cascadia,"JB-13, 2700 m","J5 (48V, 75W)",Serial,1048181-XX,"1093189-37 (EXT-xx), ODI-12-Receptacle to XSL-...",10.136.68.130:4003,ADCP 75 kHz,RDI Workhorse Long Ranger,16511.0,23006.0,Test and Development,https://jira.oceannetworks.ca/browse/EN-3580,NaN,Deploy
3,Cascadia,"JB-13, 2700 m","J10 (400V, 15A)",Ethernet,1048181-XX,TBD,NaN,BPR,"NRCan Bottom Pressure Recorder, SE",87.0,21502.0,Test and Development,https://jira.oceannetworks.ca/browse/EN-3826,NaN,Deploy
4,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone Array,icLink,NaN,50320.0,Test and Development,https://jira.oceannetworks.ca/browse/EN-3926,NaN,Deploy
5,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320.0,Test and Development,NaN,NaN,Deploy
6,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320.0,Test and Development,NaN,NaN,Deploy
7,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320.0,Test and Development,NaN,NaN,Deploy
8,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320.0,Test and Development,NaN,NaN,Deploy
9,Clayoquot,JB-08,J10 (400V 15A),Ethernet,1048181-xx,"1093189-xx, EXT-xx ,70m (already on seafloor) ...",NaN,Hydrophone Array,icLink,4042.0,48940.0,Test and Development,https://jira.oceannetworks.ca/browse/EN-3924,NaN,Deploy


In [163]:
# export to an excel file
df_out.to_excel(r'output.xlsx', index = False)